# Importing and installing required modules

In [8]:
import numpy as np
import pandas as pd
import matplotlib as plt
import folium
import json
import sklearn.cluster as kmeans
!pip install inline beautifulsoup4
!pip install inline lxml
!pip install inline requests
!pip install inline geocoder

## Importing Modules for Webscraping and for CSV

In [9]:
from bs4 import BeautifulSoup
import requests
import csv

# Scraping data using BeautifulSoup and writing the data to a csv file

In [10]:
html = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(html, "lxml")
table = soup.find("table", class_="wikitable sortable")
info = table.findAll("tr")

with open("nb.csv", "wt+", newline="") as f:
    writer = csv.writer(f)
    for i in info:
        csv_row = []
        for cell in i.findAll(["td"]):
            csv_row.append(cell.get_text())
        writer.writerow(csv_row)

# Preprocessing data using Pandas. 
#### First I imported the data from the csv. Secondly, I removed 'Not Assigned' values from Borough column
#### Thirdly, I striped unneccasary key values
#### Fourthly, I replaced the Not Assigned values in the Neighbourhood column with its equivalent Boroughs
#### Lastly, I grouped the data for duplicates so that the Postal Codes had all there assigned Nieghbourhoods in one row and sorted them 
#### by postal code

In [11]:
df = pd.read_csv('nb.csv', names = ["Postal Code", "Borough", "Neighbourhood"])
df_drop = df[df.Borough != 'Not assigned'].reset_index(drop=True)
new_df = df_drop.apply(lambda x: x.str.strip('\n') if x.dtype == "object" else x)
new_df['Neighbourhood'].replace('Not assigned', new_df.Borough, inplace = True)
new_df = new_df.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
new_df.head(11)

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


# Sample of the data to prove correctness of processing

In [12]:
new_df.sample(n=20)

,Postal Code,Borough,Neighbourhood
94,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ..."
54,M5B,Downtown Toronto,"Ryerson, Garden District"
99,M9P,Etobicoke,Westmount
33,M3N,North York,Downsview Northwest
70,M5X,Downtown Toronto,"First Canadian Place, Underground city"
79,M6L,North York,"Downsview, North Park, Upwood Park"
85,M7A,Queen's Park,Queen's Park
48,M4T,Central Toronto,"Moore Park, Summerhill East"
80,M6M,York,"Del Ray, Keelesdale, Mount Dennis, Silverthorn"
5,M1J,Scarborough,Scarborough Village


In [13]:
new_df.shape

(103, 3)

In [15]:
dh = pd.read_csv('Geospatial_Coordinates.csv')
final_df = pd.merge(new_df, dh)
final_df.head(12)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Longitude and Latitude Data